# Проект по SQL

Коронавирус застал мир врасплох, изменив привычный порядок вещей. В свободное время жители городов больше не выходят на улицу, не посещают кафе и торговые центры. Зато стало больше времени для книг. Это заметили стартаперы — и бросились создавать приложения для тех, кто любит читать.

Наша компания решила быть на волне и купила крупный сервис для чтения книг по подписке. Наша первая задача как аналитика — проанализировать базу данных.
В ней — информация о книгах, издательствах, авторах, а также пользовательские обзоры книг. 

**Цель - вывести ознакомительную информацию из базы данных для дальнейшего формулирования ценностного предложения для нового продукта**

## Описание данных

**Таблица `books`**

Содержит данные о книгах:

- `book_id` — идентификатор книги;
- `author_id` — идентификатор автора;
- `title` — название книги;
- `num_pages` — количество страниц;
- `publication_date` — дата публикации книги;
- `publisher_id` — идентификатор издателя.

**Таблица `authors`**

Содержит данные об авторах:

- `author_id` — идентификатор автора;
- `author` — имя автора.

**Таблица `publishers`**

Содержит данные об издательствах:

- `publisher_id` — идентификатор издательства;
- `publisher` — название издательства;

**Таблица `ratings`**

Содержит данные о пользовательских оценках книг:

- `rating_id` — идентификатор оценки;
- `book_id` — идентификатор книги;
- `username` — имя пользователя, оставившего оценку;
- `rating` — оценка книги.

**Таблица `reviews`**

Содержит данные о пользовательских обзорах на книги:

- `review_id` — идентификатор обзора;
- `book_id` — идентификатор книги;
- `username` — имя пользователя, написавшего обзор;
- `text` — текст обзора.

В ходе исследования мы ответим на следующие вопросы:
- Посчитаем, сколько книг вышло после 1 января 2000 года;
- Для каждой книги посчитаем количество обзоров и среднюю оценку;
- Определим издательство, которое выпустило наибольшее число книг толще 50 страниц;
- Определим автора с самой высокой средней оценкой книг — учтем только книги с 50 и более оценками;
- Посчитаем среднее количество обзоров от пользователей, которые поставили больше 50 оценок.

## Загрузка данных

Импортируем библиотеки и загрузим данные

In [1]:
import pandas as pd
from sqlalchemy import create_engine 

In [2]:
db_config = {'user': 'praktikum_student', # имя пользователя
 'pwd': 'Sdf4$2;d-d30pp', # пароль
 'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
 'port': 6432, # порт подключения
 'db': 'data-analyst-final-project-db'} # название базы данных

In [3]:
connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
 db_config['pwd'],
 db_config['host'],
 db_config['port'],
 db_config['db']) 

In [4]:
engine = create_engine(connection_string, connect_args={'sslmode':'require'})

## Исследование данных

Распечатаем первые 5 строк таблицы books

In [5]:
query_1 = '''
         SELECT *
         FROM books
         LIMIT 5
        '''
books = pd.io.sql.read_sql(query_1, con = engine)
books

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


Распечатаем первые 5 строк таблицы authors

In [6]:
query_2 = '''
         SELECT *
         FROM authors
         LIMIT 5
        '''
authors = pd.io.sql.read_sql(query_2, con = engine)
authors

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


Распечатаем первые 5 строк таблицы publishers

In [7]:
query_3 = '''
         SELECT *
         FROM publishers
         LIMIT 5
        '''
publishers = pd.io.sql.read_sql(query_3, con = engine)
publishers

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


Распечатаем первые 5 строк таблицы ratings

In [8]:
query_4 = '''
         SELECT *
         FROM ratings
         LIMIT 5
        '''
ratings = pd.io.sql.read_sql(query_4, con = engine)
ratings

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


Распечатаем первые 5 строк таблицы review

In [9]:
query_5 = '''
         SELECT *
         FROM reviews
         LIMIT 5
        '''
reviews = pd.io.sql.read_sql(query_5, con = engine)
reviews

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


Отметим, что название столбцов корректны, таблица заполнена так же корректно.

## Выполнение задания

### Посчитаем, сколько книг вышло после 1 января 2000 года

In [10]:
books

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


In [11]:
query_books = '''
         SELECT 
               COUNT(book_id)
         FROM  books
         WHERE 
               publication_date >= '01-01-2000'
        '''
books_2000 = pd.io.sql.read_sql(query_books, con = engine)
books_2000

,count
0,821


По данным таблицы books после 1 января 2000 года вышла 821 книга.

### Для каждой книги посчитаем количество обзоров и среднюю оценку

In [12]:
ratings

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


In [13]:
reviews

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


In [14]:
query_avg = '''
         SELECT
              SQB.book_id,
              SQB.title,
              AVG(SQB2.avg_rating) AS rating_avg,
              COUNT(DISTINCT SQB3.count_review) AS count_review
         FROM 
              (
              SELECT
                       books.book_id,
                       books.title
              FROM  
                       books   
              GROUP BY
                       books.book_id
                       
               ORDER BY
                      books.title
                ) AS SQB
               
                       LEFT JOIN (
                      
                                    SELECT
                                           ratings.book_id,
                                           AVG(ratings.rating) AS avg_rating
                                    FROM 
                                           ratings
                                    GROUP BY  
                                           ratings.book_id
                                    ORDER BY 
                                           avg_rating DESC  
                                    
                                  
                                    )  AS SQB2 ON SQB2.book_id = SQB.book_id
                                    
                                                    INNER JOIN (
                                                                   SELECT
                                                                         reviews.book_id,
                                                                         reviews.review_id AS count_review      
                                                                   FROM 
                                                                         reviews
                                                                   GROUP BY 
                                                                         reviews.book_id,
                                                                         count_review 
                                                                   ORDER BY 
                                                                         count_review DESC 
                                                                   
                                                                    ) AS SQB3 ON SQB3.book_id = SQB2.book_id
               
               
                                                             
        GROUP BY                        
                SQB.book_id,
                title
            
                
                
        ORDER BY
                count_review DESC,
                rating_avg DESC
                
                
        '''
reviews_and_average_rating = pd.io.sql.read_sql(query_avg, con = engine)
reviews_and_average_rating

,book_id,title,rating_avg,count_review
0,948,Twilight (Twilight #1),3.662500,7
1,302,Harry Potter and the Prisoner of Azkaban (Harr...,4.414634,6
2,299,Harry Potter and the Chamber of Secrets (Harry...,4.287500,6
3,656,The Book Thief,4.264151,6
4,734,The Glass Castle,4.206897,6
...,...,...,...,...
989,186,Death: The High Cost of Living,3.000000,1
990,446,Moo Baa La La La!,3.000000,1
991,2,1 000 Places to See Before You Die,2.500000,1
992,318,History of Beauty,2.500000,1


В таблице выше представлена информация по каждой книге (количество рецензий и средняя оценка). Для наглядности рассмотрим топ-5 книг по среднему рейтингу(с максимумом рецензий).

In [15]:
query_top5 = '''
         SELECT
               books.book_id,
               books.title AS name_book,
               COUNT(DISTINCT reviews.review_id) AS count_review,
               AVG(ratings.rating) AS avg_rating
         FROM  books
               LEFT JOIN reviews ON reviews.book_id = books.book_id
               LEFT JOIN ratings ON ratings.book_id = books.book_id      
         GROUP BY
               books.book_id
         ORDER BY
               avg_rating DESC,
               count_review DESC
         LIMIT 5      
               
        '''
top_5_rating_review = pd.io.sql.read_sql(query_top5, con = engine)
top_5_rating_review


,book_id,name_book,count_review,avg_rating
0,17,A Dirty Job (Grim Reaper #1),4,5.0
1,444,Moneyball: The Art of Winning an Unfair Game,3,5.0
2,553,School's Out—Forever (Maximum Ride #2),3,5.0
3,57,Act of Treason (Mitch Rapp #9),2,5.0
4,20,A Fistful of Charms (The Hollows #4),2,5.0


В таблице выше представлены топ-5 книг с наивысшим рейтингом и максимальным количеством рецензий.

### Определим издательство, которое выпустило наибольшее число книг толще 50 страниц

In [16]:
publishers

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


In [17]:
query_books = '''
         SELECT
               publishers.publisher_id,
               publishers.publisher AS publisher_name,
               COUNT(books.book_id) AS count_book
               
         FROM  books
               LEFT JOIN publishers ON publishers.publisher_id = books.publisher_id
         WHERE 
               books.num_pages > 50
         GROUP BY
               publisher_name,
               publishers.publisher_id
         ORDER BY 
               count_book DESC
         LIMIT 1;      
        '''
top_publishing= pd.io.sql.read_sql(query_books, con = engine)
top_publishing

,publisher_id,publisher_name,count_book
0,212,Penguin Books,42


Издательство `Penguin Books` выпустило наибольшее количество книг толще 50 листов - 42 штуки.

### Определим автора с самой высокой средней оценкой книг — учтем только книги с 50 и более оценками

In [18]:
query_top_author =  '''
          SELECT
              author,
              AVG(sub.avg_rating) AS high_avg_rating
          FROM
              (SELECT
                  author,
                  books.book_id,
                  AVG(rating) AS avg_rating
              FROM 
                  authors
              LEFT JOIN books ON books.author_id = authors.author_id
              LEFT JOIN ratings ON ratings.book_id = books.book_id
              GROUP BY
                  author,
                  books.book_id
              HAVING
                  COUNT(rating) > 50) AS sub
          GROUP BY
              author
          ORDER BY
              high_avg_rating DESC
        '''

top_author = pd.io.sql.read_sql(query_top_author, con = engine)
top_author

,author,high_avg_rating
0,J.K. Rowling/Mary GrandPré,4.283844
1,Markus Zusak/Cao Xuân Việt Khương,4.264151
2,J.R.R. Tolkien,4.258446
3,Louisa May Alcott,4.192308
4,Rick Riordan,4.080645
5,William Golding,3.901408
6,J.D. Salinger,3.825581
7,Paulo Coelho/Alan R. Clarke/Özdemir İnce,3.789474
8,William Shakespeare/Paul Werstine/Barbara A. M...,3.787879
9,Dan Brown,3.754540


Самым высокооцененным автором является Роулинг, автор Гарри Поттера.

### Посчитаем среднее количество обзоров от пользователей, которые поставили больше 50 оценок

In [19]:
query_review_avg = '''

        SELECT
            AVG(sub_review_cnt.review_cnt) AS avg_review_cnt
        FROM
            (SELECT
                COUNT(text) AS review_cnt
            FROM 
                reviews
            WHERE username IN
                (SELECT username
                FROM 
                    ratings
                GROUP BY
                    username
                HAVING
                    COUNT(ratings) > 50)
            GROUP BY 
                    username
            ) AS sub_review_cnt  
            
        '''
        
avg_review = pd.io.sql.read_sql(query_review_avg, con = engine)
avg_review

,avg_review_cnt
0,24.333333


Среднее количество обзоров от пользователей, которые поставили больше 50 оценок составляет 24 обзора

## Выводы

Проверя исследование данных и сформировав корректные запросы, мы можем сделать следующие выводы:
- После 1 января 2000 года вышла **821 книга**;
- Подготовлена таблица с количеством обзоров и средней оценкой. Топ-5 книг с наивысшей оценкой и максимальным количеством обзоров:
    - **A Dirty Job (Grim Reaper #1)**;
    - **444 Moneyball: The Art of Winning an Unfair Game**;
    - **553 School's Out—Forever (Maximum Ride #2)**;
    - **57 Act of Treason (Mitch Rapp #9)**;
    - **20 A Fistful of Charms (The Hollows #4)**.
- Издательством, которое выпустило наибольшее число книг толще 50 страниц, является **Penguin Books** с 42 книгами.
- Автор с самой высокой средней оценкой книг - **Роулинг**, автор Гарри Поттера.
- Среднее количество обзоров от пользователей, которые поставили больше 50 оценок - чуть более **24**.

Таким образом, результаты проведенного исследования позволят нашей компании наполнить сервис литературой, которая найдет отклик у пользователей и будет приносить прибыль.
